<a href="https://colab.research.google.com/github/xxKeyaxx/Multiple-Floor-Indoor-Positioning-System-using-GraphSage-and-Transfer-Learning/blob/main/Indoor_Positioning_System_(All_Floor%2C_No_TF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import pandas as pd
import numpy as np
from torch_geometric.data import Data, Batch
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

test_file_path = '/content/drive/My Drive/UTSIndoorLoc/UTS_test.csv'
train_file_path = '/content/drive/My Drive/UTSIndoorLoc/UTS_training.csv'
df_test = pd.read_csv(test_file_path)
df_train = pd.read_csv(train_file_path)
print(df_train.head())  # Preview the data

Mounted at /content/drive
   WAP001  WAP002  WAP003  WAP004  WAP005  WAP006  WAP007  WAP008  WAP009  \
0     100     100     100     100     100     100     100     100     100   
1     100     100     100     100     100     100     100     100     100   
2     100     100     100     100     100     100     100     100     100   
3     100     100     100     100     100     100     100     100     100   
4     100     100     100     100     100     100     100     100     100   

   WAP010  ...  WAP587  WAP588  WAP589    Pos_x    Pos_y  Floor_ID  \
0     100  ...     100     100     100  40.7871  31.0272         8   
1     100  ...     100     100     100  40.7871  31.0272         8   
2     100  ...     100     100     100  40.7871  31.0272         8   
3     100  ...     100     100     100  39.4464  30.7985         8   
4     100  ...     100     100     100  39.4464  30.7985         8   

   Building_ID  User_ID  Phone_type             Time  
0       New IT     root      unknow

In [3]:
def preprocess_data(data):
    # Replace 100 with 0 (no detection)
    sensor_cols = [col for col in data.columns if col.startswith('WAP')]
    data[sensor_cols] = data[sensor_cols].replace(100, 0)
    sensor_cols.append('Floor_ID')
    return data, sensor_cols

df_train, sensor_cols = preprocess_data(df_train)

In [4]:
# # Create graph data for each floor
# def create_graphs(data, sensor_cols):
#     graphs = {}
#     x = torch.tensor(data[sensor_cols].values, dtype=torch.float)
#     y = torch.tensor(data[['Pos_x', 'Pos_y']].values, dtype=torch.float)

#     # Create dummy edges (fully connected graph for simplicity)
#     edge_index = torch.combinations(torch.arange(x.size(0)), r=2).T
#     edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

#     graphs = Data(x=x, edge_index=edge_index, y=y)
#     return graphs

# graphs = create_graphs(df_train, sensor_cols)

# Create graph data for each floor
def create_graphs_by_floor(data, sensor_cols):
    graphs = {}
    for floor_id in data['Floor_ID'].unique():
        floor_data = data[data['Floor_ID'] == floor_id]
        x = torch.tensor(floor_data[sensor_cols].values, dtype=torch.float)
        y = torch.tensor(floor_data[['Pos_x', 'Pos_y']].values, dtype=torch.float)

        # Create dummy edges (fully connected graph for simplicity)
        edge_index = torch.combinations(torch.arange(x.size(0)), r=2).T
        edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

        graphs[floor_id] = Data(x=x, edge_index=edge_index, y=y)
    return graphs

graphs_by_floor = create_graphs_by_floor(df_train, sensor_cols)

In [5]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

In [6]:
import gc

# Initialize and train the model for one floor
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(in_channels=len(sensor_cols), hidden_channels=64, out_channels=2).to(device)

# Train on a single floor and fine-tune on another
def train_model(graph_data, model, epochs=100, lr=0.01):
    graph_data = graph_data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(graph_data.x, graph_data.edge_index)
        loss = criterion(pred, graph_data.y)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    return model

In [8]:
# print("Training Model")
# model = train_model(graphs, model)

max_floor = 0
for floor_id in sorted(graphs_by_floor.keys()):
    print(f"Training/Fine-tuning on Floor {floor_id}...")
    model = train_model(graphs_by_floor[floor_id], model)
    max_floor = floor_id

print("Training Complete")

Training/Fine-tuning on Floor -3...
Epoch 20, Loss: 28.5521240234375
Epoch 40, Loss: 17.637168884277344
Epoch 60, Loss: 9.52087116241455
Epoch 80, Loss: 6.833733081817627
Epoch 100, Loss: 5.337674140930176
Training/Fine-tuning on Floor -2...
Epoch 20, Loss: 47.78579330444336
Epoch 40, Loss: 24.02873420715332
Epoch 60, Loss: 16.9492244720459
Epoch 80, Loss: 12.214492797851562
Epoch 100, Loss: 9.003173828125
Training/Fine-tuning on Floor -1...
Epoch 20, Loss: 27.16414451599121
Epoch 40, Loss: 14.084573745727539
Epoch 60, Loss: 10.826127052307129
Epoch 80, Loss: 9.486392974853516
Epoch 100, Loss: 8.632204055786133
Training/Fine-tuning on Floor 0...
Epoch 20, Loss: 22.635986328125
Epoch 40, Loss: 15.206517219543457
Epoch 60, Loss: 10.50535774230957
Epoch 80, Loss: 8.976301193237305
Epoch 100, Loss: 8.048324584960938
Training/Fine-tuning on Floor 1...
Epoch 20, Loss: 35.300655364990234
Epoch 40, Loss: 17.51955223083496
Epoch 60, Loss: 12.76397705078125
Epoch 80, Loss: 9.2374906539917
Epoch 

In [11]:
from sklearn.metrics import r2_score

# Create graph data for all
def create_graphs(data, sensor_cols):
    graphs = {}
    x = torch.tensor(data[sensor_cols].values, dtype=torch.float)
    y = torch.tensor(data[['Pos_x', 'Pos_y']].values, dtype=torch.float)

    # Create dummy edges (fully connected graph for simplicity)
    edge_index = torch.combinations(torch.arange(x.size(0)), r=2).T
    edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

    graphs = Data(x=x, edge_index=edge_index, y=y)
    return graphs

graphs = create_graphs(df_train, sensor_cols)

def evaluate_model(model, test_graph):
    model.eval()
    test_graph = test_graph.to(device)
    with torch.no_grad():
        pred = model(test_graph.x, test_graph.edge_index)
        true = test_graph.y

        # Calculate Mean Absolute Error (MAE)
        mae = torch.mean(torch.abs(pred - true))

        # Calculate Root Mean Square Error (RMSE)
        rmse = torch.sqrt(torch.mean((pred - true) ** 2))

        r2 = r2_score(true.numpy(), pred.numpy())

    print("Evaluation Results:")
    print(f"MAE: {mae.item():.4f}")
    print(f"RMSE: {rmse.item():.4f}")
    print(f"R2 Score: {r2:.4f}")
    return mae.item(), rmse.item()

# Load and preprocess the new dataset
df_test, sensor_cols = preprocess_data(df_test)

# df_test = df_test[df_test['Floor_ID'] == max_floor]

# Create graph for new dataset
new_global_graph = create_graphs(df_test, sensor_cols)

# Evaluate the global model on the new graph
print("Evaluating on the new dataset...")
evaluate_model(model, new_global_graph)
print(max_floor)

Evaluating on the new dataset...
Evaluation Results:
MAE: 17.7026
RMSE: 23.3246
R2 Score: -1.3313
12
